##Training Loop (Diffusion)

In [3]:
# ==========================================
# PART 1: PREPARE DATA FROM NORMAL FOLDER
# ==========================================
import os
import glob
import cv2
import numpy as np
import torch
import torchxrayvision as xrv
from tqdm.auto import tqdm

# ⚠️ แก้ Path นี้ให้ตรงกับโฟลเดอร์ Normal ของ COVID-19 Database
RAW_DATA_PATH = r"COVID-19_Radiography_Dataset\Normal\images" 

SAVE_NPY_PATH = "x_train_normal_bestpy.npy"
IMG_SIZE = 128

# โหลด Mask Model (PSPNet) เหมือน bestpy
print("⏳ Loading PSPNet...")
seg_model = xrv.baseline_models.chestx_det.PSPNet()
seg_model.eval()

def get_bestpy_mask(img):
    # สูตร Mask เดิมจาก bestpy (L+R ไม่ลบหัวใจ)
    img = xrv.datasets.normalize(img, 255)
    if len(img.shape) == 3: img = img.mean(2) # ถ้าภาพเป็น RGB ให้แปลงเป็น Gray
    img = img[None, ...] 
    
    with torch.no_grad():
        img_tensor = torch.from_numpy(img)
        out = seg_model(img_tensor)
        pred = out[0].numpy()
    
    # รวมปอดซ้ายขวา
    mask = (pred[4] + pred[5]) > 0.5
    mask = mask.astype(np.uint8) * 255
    return cv2.resize(mask, (512, 512), interpolation=cv2.INTER_NEAREST)

def preprocess(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is None: return None
    
    # Resize ขั้นแรก
    img_512 = cv2.resize(img, (512, 512))
    
    # 1. Masking (เหมือน bestpy)
    mask = get_bestpy_mask(img_512)
    
    # 2. Enhancement (CLAHE) - เหมือน bestpy
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    img_enh = clahe.apply(img_512)
    
    # ตัดพื้นหลังทิ้ง
    img_masked = cv2.bitwise_and(img_enh, img_enh, mask=mask)
    
    # 3. Final Resize & Normalize
    img_final = cv2.resize(img_masked, (IMG_SIZE, IMG_SIZE))
    img_final = img_final.astype('float32') / 255.0
    
    return img_final

# เริ่มกระบวนการ
files = glob.glob(os.path.join(RAW_DATA_PATH, "*.png")) + glob.glob(os.path.join(RAW_DATA_PATH, "*.jpg"))
print(f"📂 Found {len(files)} images. Processing {len(files)} images...")

data = []
for f in tqdm(files[:]):
    try:
        p = preprocess(f)
        if p is not None and np.sum(p) > 0: # ไม่เอาภาพดำล้วน
            data.append(p)
    except: continue

# Save .npy
data = np.array(data)
data = np.expand_dims(data, axis=-1) # (N, 128, 128, 1)
np.save(SAVE_NPY_PATH, data)
print(f"✅ Data Saved: {SAVE_NPY_PATH} | Shape: {data.shape}")

⏳ Loading PSPNet...
📂 Found 10192 images. Processing 10192 images...


  0%|          | 0/10192 [00:00<?, ?it/s]

✅ Data Saved: x_train_normal_bestpy.npy | Shape: (10182, 128, 128, 1)


In [1]:
import torch
import torch.nn.functional as F
from diffusers import DDPMScheduler, UNet2DModel
from diffusers.optimization import get_cosine_schedule_with_warmup
from torch.utils.data import DataLoader, TensorDataset
from torch.cuda.amp import autocast, GradScaler # ⚡ ตัวช่วยเร่งความเร็ว
import numpy as np
from tqdm.auto import tqdm
import os
# ตรวจสอบ GPU
if torch.cuda.is_available():
    device = "cuda"
    print(f"✅ GPU Detected: {torch.cuda.get_device_name(0)}")
    torch.backends.cudnn.benchmark = True # ⚡ เร่งความเร็วอีกนิด
else:
    device = "cpu"
    print("⚠️ GPU not found! Training will be slow.")

✅ GPU Detected: NVIDIA GeForce RTX 4060


In [2]:
import torch
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"CUDA Version (Torch built with): {torch.version.cuda}")

PyTorch Version: 2.9.1+cu130
CUDA Available: True
CUDA Version (Torch built with): 13.0


In [1]:
# ==========================================
# PART 2: GPU TURBO TRAINING (Fix Deprecation Warning)
# ==========================================
import torch
import torch.nn.functional as F
from diffusers import DDPMScheduler, UNet2DModel
from diffusers.optimization import get_cosine_schedule_with_warmup
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from tqdm.auto import tqdm
import os

# ================= 1. CONFIG =================
DATA_PATH = "x_train_normal_bestpy.npy" 
MODEL_SAVE_DIR = "lung_diffusion_model_new"

BATCH_SIZE = 64 
IMG_SIZE = 128
EPOCHS = 5
LR = 1e-4

# ตรวจสอบ GPU
if torch.cuda.is_available():
    device = "cuda"
    print(f"✅ GPU Detected: {torch.cuda.get_device_name(0)}")
    torch.backends.cudnn.benchmark = True 
else:
    device = "cpu"
    print("⚠️ GPU not found! Training will be slow.")

# ================= 2. LOAD DATA =================
print("⏳ Loading Data...")
data = np.load(DATA_PATH)
data = np.transpose(data, (0, 3, 1, 2)) # (N, 1, 128, 128)
dataset = TensorDataset(torch.Tensor(data))

dataloader = DataLoader(
    dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    pin_memory=True if device == "cuda" else False,
    num_workers=0 # ปรับเป็น 0 เพื่อลดปัญหา IProgress ใน Windows
)

# ================= 3. SETUP MODEL =================
noise_scheduler = DDPMScheduler(num_train_timesteps=1000)

model = UNet2DModel(
    sample_size=IMG_SIZE,
    in_channels=1,
    out_channels=1,
    layers_per_block=2,
    block_out_channels=(64, 128, 256, 256),
    down_block_types=("DownBlock2D", "DownBlock2D", "AttnDownBlock2D", "DownBlock2D"),
    up_block_types=("UpBlock2D", "AttnUpBlock2D", "UpBlock2D", "UpBlock2D"),
).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
lr_scheduler = get_cosine_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=500,
    num_training_steps=len(dataloader) * EPOCHS,
)

# ⚡ แก้ไขตรงนี้: ใช้ torch.amp.GradScaler แทน torch.cuda.amp.GradScaler
scaler = torch.amp.GradScaler('cuda')

# ================= 4. TRAINING LOOP =================
print("🚀 Start GPU Turbo Training...")

for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0.0
    # ใช้ tqdm ธรรมดาเพื่อเลี่ยงปัญหา IProgress
    progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{EPOCHS}", leave=False)
    
    for batch in progress_bar:
        clean_images = batch[0].to(device, non_blocking=True)
        
        # 1. Add Noise
        noise = torch.randn(clean_images.shape).to(device)
        bs = clean_images.shape[0]
        timesteps = torch.randint(0, 1000, (bs,), device=device).long()
        noisy_images = noise_scheduler.add_noise(clean_images, noise, timesteps)
        
        optimizer.zero_grad()
        
        # 2. Mixed Precision Forward Pass (แก้ไข Syntax ใหม่)
        with torch.amp.autocast('cuda'):
            noise_pred = model(noisy_images, timesteps).sample
            loss = F.mse_loss(noise_pred, noise)
        
        # 3. Backward Pass with Scaler
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        lr_scheduler.step()
        
        epoch_loss += loss.item()
        progress_bar.set_postfix({"Loss": loss.item()})
        
    avg_loss = epoch_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{EPOCHS} | Avg Loss: {avg_loss:.5f}")

# Save
model.save_pretrained(MODEL_SAVE_DIR)
print(f"💾 Model Saved to: {MODEL_SAVE_DIR}")
print("✅ Training Complete! Ready for Inference.")

✅ GPU Detected: NVIDIA GeForce RTX 4060
⏳ Loading Data...
🚀 Start GPU Turbo Training...


Epoch 1/5:   0%|          | 0/160 [00:00<?, ?it/s]

Epoch 1/5 | Avg Loss: 0.46997


Epoch 2/5:   0%|          | 0/160 [00:00<?, ?it/s]

Epoch 2/5 | Avg Loss: 0.03531


...

Epoch 3/5 | Avg Loss: 0.01796


Epoch 4/5:   0%|          | 0/160 [00:00<?, ?it/s]

Epoch 4/5 | Avg Loss: 0.01251


Epoch 5/5:   0%|          | 0/160 [00:00<?, ?it/s]

Epoch 5/5 | Avg Loss: 0.01094
💾 Model Saved to: lung_diffusion_model_new
✅ Training Complete! Ready for Inference.


##Use 

In [ ]:
import os
import glob
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torchxrayvision as xrv
from diffusers import DDPMScheduler, UNet2DModel
from tqdm.auto import tqdm
import warnings

warnings.filterwarnings("ignore")
plt.rcParams['font.family'] = 'Tahoma'

device = "cuda" if torch.cuda.is_available() else "cpu"

# ================= 1. CONFIGURATION =================
class Config:
    ROOT_DIR = "Chest xray CP class"
    MODEL_PATH = "lung_diffusion_model"
    
    BASE_OUTPUT = "Final_User_Code_Integration"
    DIRS = {
        "heatmaps": os.path.join(BASE_OUTPUT, "Heatmaps"),
        "timelines": os.path.join(BASE_OUTPUT, "Timelines"),
        "comparisons": os.path.join(BASE_OUTPUT, "Comparisons"),
        "csv": os.path.join(BASE_OUTPUT, "Clinical_Data")
    }
    
    # ค่าเหล่านี้จะถูก Override ในฟังก์ชัน analyze_lesion_score ตามโค้ดคุณ
    IMG_SIZE = 128
    START_TIMESTEP = 180 
    THRESHOLD = 0.15
    SEED = 42

for d in Config.DIRS.values():
    if not os.path.exists(d): os.makedirs(d)

# ================= 2. MODEL LOADING =================
print("⏳ Loading Models...")
seg_model = xrv.baseline_models.chestx_det.PSPNet()
seg_model.eval()

if os.path.exists(Config.MODEL_PATH):
    noise_scheduler = DDPMScheduler(num_train_timesteps=1000)
    diffusion_model = UNet2DModel.from_pretrained(Config.MODEL_PATH).to(device)
    diffusion_model.eval()
    print("✅ Models Ready!")
else:
    raise FileNotFoundError("❌ ไม่พบโมเดล Diffusion")

# ================= 3. HELPER FUNCTIONS (Mask & Enhance) =================
# ใช้ Logic เดิมในการเตรียมภาพเบื้องต้น (Mask L+R)
def get_masks(img_numpy_uint8):
    try:
        h, w = img_numpy_uint8.shape
        img_norm = xrv.datasets.normalize(img_numpy_uint8, 255) 
        img_resized = cv2.resize(img_norm, (512, 512))
        img_tensor = torch.from_numpy(img_resized)[None, None, ...].float()
        with torch.no_grad(): outputs = seg_model(img_tensor)
        pred = outputs[0].numpy()
        def resize_m(m): return cv2.resize(m, (w, h), interpolation=cv2.INTER_LINEAR)
        
        mask_l = (resize_m(pred[4]) > 0.5).astype(np.uint8) * 255
        mask_r = (resize_m(pred[5]) > 0.5).astype(np.uint8) * 255
        mask_total = cv2.bitwise_or(mask_l, mask_r)
        
        return mask_total, mask_l, mask_r
    except:
        z = np.zeros(img_numpy_uint8.shape, dtype=np.uint8)
        return z, z, z

def enhance_lung_clarity(img_gray, lung_mask):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    enhanced = clahe.apply(img_gray)
    return cv2.bitwise_and(enhanced, enhanced, mask=lung_mask)

def process_image(img_path):
    if not os.path.exists(img_path): return None, None
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is None: return None, None
    
    mask_total, _, _ = get_masks(img)
    if np.sum(mask_total) == 0: enhanced = img
    else: enhanced = enhance_lung_clarity(img, mask_total)
        
    resized = cv2.resize(enhanced, (Config.IMG_SIZE, Config.IMG_SIZE))
    img_tensor = torch.tensor(resized).float() / 255.0
    
    # Return Tensor and Original Numpy (for visualization)
    return img_tensor.unsqueeze(0).unsqueeze(0).to(device), img

# ================= 4. CORE FUNCTION (FROM YOUR CODE) =================
def analyze_lesion_score(img_path, model, scheduler, start_timestep=180, threshold=0.15, seed=42):
    """
    คืนค่า 4 อย่าง: Overlay, Score, Original Image, AI Reconstructed Image
    """
    # 1. เตรียมภาพ
    input_tensor, original_img_np = process_image(img_path)
    if input_tensor is None: return None, 0.0, None, None

    # --- 🔒 ล็อกค่าสุ่ม (เพื่อความนิ่ง) ---
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

    # 2. ใส่ Noise 
    noise = torch.randn(input_tensor.shape).to(device)
    timesteps = torch.tensor([start_timestep], device=device).long()
    noisy_image = scheduler.add_noise(input_tensor, noise, timesteps)

    # 3. ซ่อมภาพ
    current_image = noisy_image
    scheduler_timesteps = scheduler.timesteps
    start_index = (scheduler_timesteps == start_timestep).nonzero(as_tuple=True)[0].item()
    subset_timesteps = scheduler_timesteps[start_index:]

    for t in subset_timesteps:
        with torch.no_grad():
            model_output = model(current_image, t).sample
            current_image = scheduler.step(model_output, t, current_image).prev_sample

    # 4. เตรียมข้อมูลภาพ
    img_recon = current_image.cpu().numpy()[0, 0]
    img_orig = input_tensor.cpu().numpy()[0, 0]
    
    # แปลงช่วงข้อมูลเป็น 0-1
    img_orig = (img_orig + 1) / 2
    img_recon = (img_recon - img_recon.min()) / (img_recon.max() - img_recon.min())
    
    # Post-process
    img_orig_blur = cv2.GaussianBlur(img_orig, (3, 3), 0)
    diff_clean = np.abs(img_orig_blur - img_recon)
    
    # Clean Background & Ribs
    background_mask = np.where(img_orig < 0.1, 0, 1) 
    diff_clean = diff_clean * background_mask 
    lesion_map = np.where(diff_clean > threshold, diff_clean, 0)

    # 5. คำนวณคะแนน
    lesion_pixels = np.count_nonzero(lesion_map)
    lung_pixels = np.count_nonzero(img_orig > 0.1)
    
    if lung_pixels == 0: score = 0.0
    else: score = (lesion_pixels / lung_pixels) * 100

    # 6. สร้างภาพ Overlay
    heatmap_display = np.clip(lesion_map * 3, 0, 1)
    heatmap_color = cv2.applyColorMap((heatmap_display * 255).astype(np.uint8), cv2.COLORMAP_JET)
    orig_bgr = cv2.cvtColor((img_orig * 255).astype(np.uint8), cv2.COLOR_GRAY2BGR)
    
    mask_lesion = (heatmap_display > 0).astype(np.uint8) * 255
    bg = cv2.bitwise_and(orig_bgr, orig_bgr, mask=cv2.bitwise_not(mask_lesion))
    fg = cv2.bitwise_and(heatmap_color, heatmap_color, mask=mask_lesion)
    overlay = cv2.add(bg, fg)

    # ⚡ คืนค่าภาพต้นฉบับและภาพ AI กลับไปด้วย
    return overlay, score, img_orig, img_recon

# ================= 5. COMPARISON FUNCTION (FROM YOUR CODE - ADAPTED FOR LOOP) =================
def generate_comparison_plot(res1, res2, pid, cls_name, dates):
    # Unpack Data
    overlay1, score1, orig1, recon1 = res1
    overlay2, score2, orig2, recon2 = res2
    
    # คำนวณผล
    diff_score = score2 - score1
    if diff_score < -1.0:
        status = "✅ อาการดีขึ้น (Improved)"
        color = 'green'
    elif diff_score > 1.0:
        status = "❌ อาการแย่ลง (Worsened)"
        color = 'red'
    else:
        status = "⚖️ อาการทรงตัว (Stable)"
        color = 'blue'

    # ================= 🖼️ แสดงผลแบบตาราง 2x3 =================
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    # --- แถวที่ 1: Day 1 ---
    axes[0, 0].imshow(orig1, cmap='gray')
    axes[0, 0].set_title(f"Day 1 ({dates[0]}): Original", fontsize=12)
    
    axes[0, 1].imshow(recon1, cmap='gray')
    axes[0, 1].set_title("Day 1: AI Healed", fontsize=12)
    
    axes[0, 2].imshow(cv2.cvtColor(overlay1, cv2.COLOR_BGR2RGB))
    axes[0, 2].set_title(f"Day 1 Result: {score1:.2f}%", fontsize=12, fontweight='bold', color='blue')
    
    # --- แถวที่ 2: Day 2 ---
    axes[1, 0].imshow(orig2, cmap='gray')
    axes[1, 0].set_title(f"Day 2 ({dates[1]}): Original", fontsize=12)
    
    axes[1, 1].imshow(recon2, cmap='gray')
    axes[1, 1].set_title("Day 2: AI Healed", fontsize=12)
    
    axes[1, 2].imshow(cv2.cvtColor(overlay2, cv2.COLOR_BGR2RGB))
    axes[1, 2].set_title(f"Day 2 Result: {score2:.2f}%", fontsize=12, fontweight='bold', color='blue')
    
    for ax in axes.flat: ax.axis('off')

    plt.suptitle(f"Patient {pid} Progress: {status} ({diff_score:+.2f}%)", fontsize=16, color=color, y=0.96)
    plt.tight_layout()
    
    # Save Plot
    save_path = os.path.join(Config.DIRS["comparisons"], f"{cls_name}_{pid}_Compare.jpg")
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.close()

# ================= 6. EXECUTION LOOP (Bestpy Copy 2 Structure) =================
def get_timeline_files(folder):
    files = glob.glob(os.path.join(folder, "*.jpg"))
    daily = {}
    for f in files:
        try:
            parts = os.path.basename(f).replace(".jpg", "").split("_")
            d = datetime.strptime(parts[-2], "%Y%m%d")
            s = int(parts[-1])
            if d not in daily or s > daily[d][0]: daily[d] = (s, f)
        except: continue
    return [daily[d][1] for d in sorted(daily)], sorted(daily)

from datetime import datetime

all_data = []
print("🚀 Starting Analysis with YOUR Custom Function...")

for cls in ["novap", "vap"]:
    c_path = os.path.join(Config.ROOT_DIR, cls)
    if not os.path.exists(c_path): continue
    
    for pid in os.listdir(c_path):
        p_path = os.path.join(c_path, pid)
        files, dates = get_timeline_files(p_path)
        if not files: continue
        
        print(f"Processing: {pid} ({len(files)} scans)")
        patient_results = [] # Store raw results (overlay, score, orig, recon)
        patient_dates = []
        
        # 1. Analyze Each Image
        for i, f in enumerate(files):
            date_s = dates[i].strftime("%Y-%m-%d")
            
            # Call YOUR function
            res = analyze_lesion_score(f, diffusion_model, noise_scheduler, 
                                       start_timestep=Config.START_TIMESTEP, 
                                       threshold=Config.THRESHOLD,
                                       seed=Config.SEED)
            
            overlay, score, orig, recon = res
            
            if overlay is not None:
                patient_results.append(res)
                patient_dates.append(date_s)
                
                # Save Heatmap
                fname = f"{pid}_{date_s}.jpg"
                save_dir = os.path.join(Config.DIRS["heatmaps"], cls, pid)
                if not os.path.exists(save_dir): os.makedirs(save_dir)
                cv2.imwrite(os.path.join(save_dir, fname), overlay)
                
                # Collect Data for CSV
                all_data.append({
                    "Patient_ID": pid,
                    "Date": date_s,
                    "Percentage of opacity": round(score, 2),
                    "Group": cls
                })

        # 2. Compare First vs Last (using YOUR comparison logic)
        if len(patient_results) >= 2:
            generate_comparison_plot(patient_results[0], patient_results[-1], pid, cls, [patient_dates[0], patient_dates[-1]])
        
        # 3. Timeline Plot (Optional - from copy 2)
        if len(patient_results) > 0:
            scores = [d["Percentage of opacity"] for d in all_data if d["Patient_ID"] == pid]
            
            plt.figure(figsize=(4 * len(patient_results), 8))
            for i in range(len(patient_results)):
                ax = plt.subplot(2, len(patient_results), i+1)
                ax.imshow(cv2.cvtColor(patient_results[i][0], cv2.COLOR_BGR2RGB)) # [0] is overlay
                color = 'red' if scores[i] > 20 else 'green'
                ax.set_title(f"{patient_dates[i]}\nOp: {scores[i]}%", color=color, fontweight='bold')
                ax.axis('off')
            ax2 = plt.subplot(2, 1, 2)
            ax2.plot(patient_dates, scores, 'b-o', linewidth=2)
            ax2.set_ylim(0, 100); ax2.grid(True, alpha=0.3)
            plt.savefig(os.path.join(Config.DIRS["timelines"], f"{cls}_{pid}_Timeline.jpg"), bbox_inches='tight')
            plt.close()

# Save CSV
if all_data:
    df = pd.DataFrame(all_data)
    csv_path = os.path.join(Config.DIRS["csv"], "Clinical_Data.csv")
    df.to_csv(csv_path, index=False)
    print(f"\n✅ Final Report Saved: {csv_path}")

⏳ Loading Models...
✅ Models Ready!
🚀 Starting Analysis with YOUR Custom Function...
Processing: 2268 (10 scans)
Processing: 2272 (18 scans)
Processing: 2273 (4 scans)
Processing: 2278 (5 scans)
Processing: 2279 (6 scans)
Processing: 2283 (6 scans)
Processing: 2285 (5 scans)
Processing: 2286 (9 scans)
Processing: 2288 (5 scans)
Processing: 2289 (6 scans)
Processing: 2290 (13 scans)
Processing: 2299 (2 scans)
Processing: 2300 (10 scans)
Processing: 2303 (3 scans)
Processing: 2304 (6 scans)
Processing: 2310 (1 scans)
Processing: 2312 (5 scans)
Processing: 2315 (5 scans)
Processing: 2316 (2 scans)
Processing: 2317 (7 scans)
Processing: 2320 (2 scans)
Processing: 2321 (9 scans)
Processing: 2328 (8 scans)
Processing: 2336 (6 scans)
Processing: 2337 (4 scans)
Processing: 2338 (4 scans)
Processing: 2131 (7 scans)
Processing: 2140 (25 scans)
Processing: 2157 (18 scans)
Processing: 2167 (14 scans)
Processing: 2179 (26 scans)
Processing: 2199 (14 scans)
Processing: 2200 (13 scans)
Processing: 22

: 

In [1]:
import os
import glob
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torchxrayvision as xrv
from diffusers import DDPMScheduler, UNet2DModel
from tqdm.auto import tqdm
import warnings

warnings.filterwarnings("ignore")
plt.rcParams['font.family'] = 'Tahoma'

device = "cuda" if torch.cuda.is_available() else "cpu"

# ================= 1. CONFIGURATION =================
class Config:
    ROOT_DIR = "Chest xray CP class"
    MODEL_PATH = "lung_diffusion_model"
    
    # ตั้งชื่อ Folder output ใหม่ให้ชัดเจน
    BASE_OUTPUT = "Final_User_Code_MaroonBG_800000" 
    DIRS = {
        "heatmaps": os.path.join(BASE_OUTPUT, "Heatmaps"),
        "timelines": os.path.join(BASE_OUTPUT, "Timelines"),
        "comparisons": os.path.join(BASE_OUTPUT, "Comparisons"),
        "csv": os.path.join(BASE_OUTPUT, "Clinical_Data")
    }
    
    IMG_SIZE = 128
    START_TIMESTEP = 180 
    THRESHOLD = 0.15 
    SEED = 42

for d in Config.DIRS.values():
    if not os.path.exists(d): os.makedirs(d)

# ================= 2. MODEL LOADING =================
print("⏳ Loading Models...")
seg_model = xrv.baseline_models.chestx_det.PSPNet()
seg_model.eval()

if os.path.exists(Config.MODEL_PATH):
    noise_scheduler = DDPMScheduler(num_train_timesteps=1000)
    diffusion_model = UNet2DModel.from_pretrained(Config.MODEL_PATH).to(device)
    diffusion_model.eval()
    print("✅ Models Ready!")
else:
    raise FileNotFoundError("❌ ไม่พบโมเดล Diffusion")

# ================= 3. HELPER FUNCTIONS (Mask & Enhance) =================
def get_masks(img_numpy_uint8):
    try:
        h, w = img_numpy_uint8.shape
        img_norm = xrv.datasets.normalize(img_numpy_uint8, 255) 
        img_resized = cv2.resize(img_norm, (512, 512))
        img_tensor = torch.from_numpy(img_resized)[None, None, ...].float()
        with torch.no_grad(): outputs = seg_model(img_tensor)
        pred = outputs[0].numpy()
        def resize_m(m): return cv2.resize(m, (w, h), interpolation=cv2.INTER_LINEAR)
        
        mask_l = (resize_m(pred[4]) > 0.5).astype(np.uint8) * 255
        mask_r = (resize_m(pred[5]) > 0.5).astype(np.uint8) * 255
        mask_total = cv2.bitwise_or(mask_l, mask_r)
        
        return mask_total, mask_l, mask_r
    except:
        z = np.zeros(img_numpy_uint8.shape, dtype=np.uint8)
        return z, z, z

def enhance_lung_clarity(img_gray, lung_mask):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    enhanced = clahe.apply(img_gray)
    return cv2.bitwise_and(enhanced, enhanced, mask=lung_mask)

def process_image(img_path):
    if not os.path.exists(img_path): return None, None, None
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is None: return None, None, None
    
    mask_total, _, _ = get_masks(img)
    
    mask_resized = cv2.resize(mask_total, (Config.IMG_SIZE, Config.IMG_SIZE), interpolation=cv2.INTER_NEAREST)
    mask_binary = (mask_resized > 127).astype(np.float32) 

    if np.sum(mask_total) == 0: enhanced = img
    else: enhanced = enhance_lung_clarity(img, mask_total)
        
    resized = cv2.resize(enhanced, (Config.IMG_SIZE, Config.IMG_SIZE))
    img_tensor = torch.tensor(resized).float() / 255.0
    
    return img_tensor.unsqueeze(0).unsqueeze(0).to(device), img, mask_binary

# ================= 4. CORE FUNCTION (UPDATED: HEX #800000 BG) =================
def analyze_lesion_score(img_path, model, scheduler, start_timestep=180, threshold=0.15, seed=42):
    # 1. เตรียมภาพและ Mask
    input_tensor, original_img_np, mask_binary = process_image(img_path)
    if input_tensor is None: return None, 0.0, None, None

    # --- 🔒 ล็อกค่าสุ่ม ---
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

    # 2. ใส่ Noise 
    noise = torch.randn(input_tensor.shape).to(device)
    timesteps = torch.tensor([start_timestep], device=device).long()
    noisy_image = scheduler.add_noise(input_tensor, noise, timesteps)

    # 3. ซ่อมภาพ
    current_image = noisy_image
    scheduler_timesteps = scheduler.timesteps
    start_index = (scheduler_timesteps == start_timestep).nonzero(as_tuple=True)[0].item()
    subset_timesteps = scheduler_timesteps[start_index:]

    for t in subset_timesteps:
        with torch.no_grad():
            model_output = model(current_image, t).sample
            current_image = scheduler.step(model_output, t, current_image).prev_sample

    # 4. เตรียมข้อมูลภาพ
    img_recon = current_image.cpu().numpy()[0, 0]
    img_orig = input_tensor.cpu().numpy()[0, 0]
    
    img_orig = (img_orig + 1) / 2
    img_recon = (img_recon - img_recon.min()) / (img_recon.max() - img_recon.min())
    
    img_orig_blur = cv2.GaussianBlur(img_orig, (3, 3), 0)
    diff_raw = np.abs(img_orig_blur - img_recon)
    
    # Scoring (MAE inside mask)
    diff_inside_lung = diff_raw * mask_binary
    lung_pixel_count = np.sum(mask_binary)
    if lung_pixel_count == 0: score = 0.0
    else: score = (np.sum(diff_inside_lung) / lung_pixel_count) * 100

    lesion_map = np.where(diff_inside_lung > threshold, diff_inside_lung, 0)

    # 5. สร้างภาพ Overlay
    heatmap_display = np.clip(lesion_map * 3, 0, 1)
    heatmap_color = cv2.applyColorMap((heatmap_display * 255).astype(np.uint8), cv2.COLORMAP_JET)
    orig_bgr = cv2.cvtColor((img_orig * 255).astype(np.uint8), cv2.COLOR_GRAY2BGR)
    
    mask_lesion = (heatmap_display > 0).astype(np.uint8) * 255
    bg = cv2.bitwise_and(orig_bgr, orig_bgr, mask=cv2.bitwise_not(mask_lesion))
    fg = cv2.bitwise_and(heatmap_color, heatmap_color, mask=mask_lesion)
    overlay = cv2.add(bg, fg)

    # --- 🔴🔴 จุดที่แก้ไข: ถมสีแดงเข้ม (#800000) นอก Mask 🔴🔴 ---
    # mask_binary คือ 1.0 ในปอด, 0.0 นอกปอด
    outside_mask = (mask_binary == 0)
    
    # เปลี่ยนพิกเซลนอกปอดให้เป็นสีตาม HEX #800000
    # OpenCV ใช้ระบบ BGR ดังนั้น #800000 (RGB) คือ B=0, G=0, R=128 (0x80 ฐานสิบคือ 128)
    overlay[outside_mask, 0] = 0   # Blue Channel
    overlay[outside_mask, 1] = 0   # Green Channel
    overlay[outside_mask, 2] = 128 # Red Channel (128 decimal = 80 hex)
    # -----------------------------------------------------------

    return overlay, score, img_orig, img_recon

# ================= 5. COMPARISON FUNCTION =================
def generate_comparison_plot(res1, res2, pid, cls_name, dates):
    overlay1, score1, orig1, recon1 = res1
    overlay2, score2, orig2, recon2 = res2
    
    diff_score = score2 - score1
    
    if diff_score < 0: status = "✅ อาการดีขึ้น (Improved)"; color = 'green'
    elif diff_score > 0: status = "❌ อาการแย่ลง (Worsened)"; color = 'red'
    else: status = "⚖️ อาการทรงตัว (Stable)"; color = 'blue'

    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    # Day 1
    axes[0, 0].imshow(orig1, cmap='gray'); axes[0, 0].set_title(f"Day 1 ({dates[0]})", fontsize=12)
    axes[0, 1].imshow(recon1, cmap='gray'); axes[0, 1].set_title("Day 1: AI Healed", fontsize=12)
    # แสดงภาพ Overlay (พื้นหลังแดงเข้ม #800000)
    axes[0, 2].imshow(cv2.cvtColor(overlay1, cv2.COLOR_BGR2RGB)) 
    axes[0, 2].set_title(f"Score: {score1:.2f} (MAE)", fontsize=12, fontweight='bold', color='blue')
    
    # Day 2
    axes[1, 0].imshow(orig2, cmap='gray'); axes[1, 0].set_title(f"Day 2 ({dates[1]})", fontsize=12)
    axes[1, 1].imshow(recon2, cmap='gray'); axes[1, 1].set_title("Day 2: AI Healed", fontsize=12)
    # แสดงภาพ Overlay (พื้นหลังแดงเข้ม #800000)
    axes[1, 2].imshow(cv2.cvtColor(overlay2, cv2.COLOR_BGR2RGB)) 
    axes[1, 2].set_title(f"Score: {score2:.2f} (MAE)", fontsize=12, fontweight='bold', color='blue')
    
    for ax in axes.flat: ax.axis('off')
    plt.suptitle(f"Patient {pid} Progress: {status} (Diff: {diff_score:+.2f})", fontsize=16, color=color, y=0.96)
    plt.tight_layout()
    
    save_path = os.path.join(Config.DIRS["comparisons"], f"{cls_name}_{pid}_Compare.jpg")
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.close()

# ================= 6. EXECUTION LOOP =================
def get_timeline_files(folder):
    files = glob.glob(os.path.join(folder, "*.jpg"))
    daily = {}
    for f in files:
        try:
            parts = os.path.basename(f).replace(".jpg", "").split("_")
            d = datetime.strptime(parts[-2], "%Y%m%d")
            s = int(parts[-1])
            if d not in daily or s > daily[d][0]: daily[d] = (s, f)
        except: continue
    return [daily[d][1] for d in sorted(daily)], sorted(daily)

from datetime import datetime

all_data = []
print("🚀 Starting Analysis with HEX #800000 BACKGROUND outside mask...")

for cls in ["novap", "vap"]:
    c_path = os.path.join(Config.ROOT_DIR, cls)
    if not os.path.exists(c_path): continue
    
    for pid in os.listdir(c_path):
        p_path = os.path.join(c_path, pid)
        files, dates = get_timeline_files(p_path)
        if not files: continue
        
        print(f"Processing: {pid} ({len(files)} scans)")
        patient_results = []
        patient_dates = []
        
        # 1. Analyze Each Image
        for i, f in enumerate(files):
            date_s = dates[i].strftime("%Y-%m-%d")
            
            res = analyze_lesion_score(f, diffusion_model, noise_scheduler, 
                                       start_timestep=Config.START_TIMESTEP, 
                                       threshold=Config.THRESHOLD,
                                       seed=Config.SEED)
            
            overlay, score, orig, recon = res
            
            if overlay is not None:
                patient_results.append(res)
                patient_dates.append(date_s)
                
                # Save Heatmap
                fname = f"{pid}_{date_s}.jpg"
                save_dir = os.path.join(Config.DIRS["heatmaps"], cls, pid)
                if not os.path.exists(save_dir): os.makedirs(save_dir)
                cv2.imwrite(os.path.join(save_dir, fname), overlay)
                
                all_data.append({
                    "Patient_ID": pid,
                    "Date": date_s,
                    "Opacity_Score_MAE": round(score, 2),
                    "Group": cls
                })

        # 2. Compare First vs Last
        if len(patient_results) >= 2:
            generate_comparison_plot(patient_results[0], patient_results[-1], pid, cls, [patient_dates[0], patient_dates[-1]])
        
        # 3. Timeline Plot
        if len(patient_results) > 0:
            scores = [d["Opacity_Score_MAE"] for d in all_data if d["Patient_ID"] == pid]
            
            plt.figure(figsize=(4 * len(patient_results), 8))
            for i in range(len(patient_results)):
                ax = plt.subplot(2, len(patient_results), i+1)
                # แสดงภาพ Overlay (พื้นหลังแดงเข้ม #800000)
                ax.imshow(cv2.cvtColor(patient_results[i][0], cv2.COLOR_BGR2RGB))
                
                current_score = scores[i]
                if i == 0: color = 'blue'
                else:
                    prev_score = scores[i-1]
                    if current_score < prev_score: color = 'green'
                    elif current_score > prev_score: color = 'red'
                    else: color = 'blue'
                
                ax.set_title(f"{patient_dates[i]}\nScore: {current_score}", color=color, fontweight='bold', fontsize=14)
                ax.axis('off')
            
            ax2 = plt.subplot(2, 1, 2)
            ax2.plot(patient_dates, scores, 'b-o', linewidth=2)
            ax2.set_ylabel("Opacity Score (MAE)")
            ax2.grid(True, alpha=0.3)
            
            plt.suptitle(f"Timeline: {pid} ({cls})", fontsize=16)
            plt.savefig(os.path.join(Config.DIRS["timelines"], f"{cls}_{pid}_Timeline.jpg"), bbox_inches='tight')
            plt.close()

# Save CSV
if all_data:
    df = pd.DataFrame(all_data)
    csv_path = os.path.join(Config.DIRS["csv"], "Clinical_Data.csv")
    df.to_csv(csv_path, index=False)
    print(f"\n✅ Final Report Saved: {csv_path}")

⏳ Loading Models...
✅ Models Ready!
🚀 Starting Analysis with HEX #800000 BACKGROUND outside mask...
Processing: 2268 (10 scans)
Processing: 2272 (18 scans)
Processing: 2273 (4 scans)
Processing: 2278 (5 scans)
Processing: 2279 (6 scans)
Processing: 2283 (6 scans)
Processing: 2285 (5 scans)
Processing: 2286 (9 scans)
Processing: 2288 (5 scans)
Processing: 2289 (6 scans)
Processing: 2290 (13 scans)
Processing: 2299 (2 scans)
Processing: 2300 (10 scans)
Processing: 2303 (3 scans)
Processing: 2304 (6 scans)
Processing: 2310 (1 scans)
Processing: 2312 (5 scans)
Processing: 2315 (5 scans)
Processing: 2316 (2 scans)
Processing: 2317 (7 scans)
Processing: 2320 (2 scans)
Processing: 2321 (9 scans)
Processing: 2328 (8 scans)
Processing: 2336 (6 scans)
Processing: 2337 (4 scans)
Processing: 2338 (4 scans)
Processing: 2131 (7 scans)
Processing: 2140 (25 scans)
Processing: 2157 (18 scans)
Processing: 2167 (14 scans)
Processing: 2179 (26 scans)
Processing: 2199 (14 scans)
Processing: 2200 (13 scans)

In [ ]:
import os
import glob
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torchxrayvision as xrv
from diffusers import DDPMScheduler, UNet2DModel
from tqdm.auto import tqdm
import warnings

warnings.filterwarnings("ignore")
plt.rcParams['font.family'] = 'Tahoma'

device = "cuda" if torch.cuda.is_available() else "cpu"

# ================= 1. CONFIGURATION =================
class Config:
    ROOT_DIR = "Chest xray CP class"
    MODEL_PATH = "lung_diffusion_model"
    
    BASE_OUTPUT = "Final_User_Code_FullMetrics_CSV" 
    DIRS = {
        "heatmaps": os.path.join(BASE_OUTPUT, "Heatmaps"),
        "timelines": os.path.join(BASE_OUTPUT, "Timelines"),
        "comparisons": os.path.join(BASE_OUTPUT, "Comparisons"),
        "csv": os.path.join(BASE_OUTPUT, "Clinical_Data")
    }
    
    IMG_SIZE = 128
    START_TIMESTEP = 180 
    THRESHOLD = 0.15 
    HIGH_OPACITY_THRESHOLD = 0.40 # ค่าสมมติสำหรับ High Opacity ในงานวิจัย (เช่น Consolidation)
    SEED = 42

for d in Config.DIRS.values():
    if not os.path.exists(d): os.makedirs(d)

# ================= 2. MODEL LOADING =================
print("⏳ Loading Models...")
seg_model = xrv.baseline_models.chestx_det.PSPNet()
seg_model.eval()

if os.path.exists(Config.MODEL_PATH):
    noise_scheduler = DDPMScheduler(num_train_timesteps=1000)
    diffusion_model = UNet2DModel.from_pretrained(Config.MODEL_PATH).to(device)
    diffusion_model.eval()
    print("✅ Models Ready!")
else:
    raise FileNotFoundError("❌ ไม่พบโมเดล Diffusion")

# ================= 3. HELPER FUNCTIONS =================
def get_masks(img_numpy_uint8):
    try:
        h, w = img_numpy_uint8.shape
        img_norm = xrv.datasets.normalize(img_numpy_uint8, 255) 
        img_resized = cv2.resize(img_norm, (512, 512))
        img_tensor = torch.from_numpy(img_resized)[None, None, ...].float()
        with torch.no_grad(): outputs = seg_model(img_tensor)
        pred = outputs[0].numpy()
        def resize_m(m): return cv2.resize(m, (w, h), interpolation=cv2.INTER_LINEAR)
        
        # Mask แยกซ้ายขวา
        mask_l = (resize_m(pred[4]) > 0.5).astype(np.uint8) * 255
        mask_r = (resize_m(pred[5]) > 0.5).astype(np.uint8) * 255
        mask_total = cv2.bitwise_or(mask_l, mask_r)
        
        return mask_total, mask_l, mask_r
    except:
        z = np.zeros(img_numpy_uint8.shape, dtype=np.uint8)
        return z, z, z

def enhance_lung_clarity(img_gray, lung_mask):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    enhanced = clahe.apply(img_gray)
    return cv2.bitwise_and(enhanced, enhanced, mask=lung_mask)

def process_image(img_path):
    if not os.path.exists(img_path): return None, None, None, None, None
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is None: return None, None, None, None, None
    
    # ได้ Mask รวม, Mask ซ้าย, Mask ขวา
    mask_total, mask_l, mask_r = get_masks(img)
    
    # Resize Masks ให้เท่ากับ IMG_SIZE (128)
    def resize_binary(m):
        m_res = cv2.resize(m, (Config.IMG_SIZE, Config.IMG_SIZE), interpolation=cv2.INTER_NEAREST)
        return (m_res > 127).astype(np.float32)

    mask_binary_total = resize_binary(mask_total)
    mask_binary_l = resize_binary(mask_l)
    mask_binary_r = resize_binary(mask_r)

    if np.sum(mask_total) == 0: enhanced = img
    else: enhanced = enhance_lung_clarity(img, mask_total)
        
    resized = cv2.resize(enhanced, (Config.IMG_SIZE, Config.IMG_SIZE))
    img_tensor = torch.tensor(resized).float() / 255.0
    
    return img_tensor.unsqueeze(0).unsqueeze(0).to(device), img, mask_binary_total, mask_binary_l, mask_binary_r

# ฟังก์ชันคำนวณ Stat รายปอด (Mapping CT metrics -> X-ray metrics)
def calculate_lung_metrics(diff_map, mask_binary, img_orig_norm):
    """
    diff_map: แผนที่ความผิดปกติ (Difference Map)
    mask_binary: พื้นที่ปอด (0 หรือ 1)
    img_orig_norm: ภาพต้นฉบับ (0.0 - 1.0) ใช้หา Intensity (แทน HU)
    """
    lung_pixels = np.sum(mask_binary)
    
    if lung_pixels == 0:
        return {
            "Affected": "No",
            "Opacity Score": 0,
            "Lung Volume (px)": 0,
            "Volume of Opacity (px)": 0,
            "% Opacity": 0.0,
            "Volume High Opacity (px)": 0,
            "% High Opacity": 0.0,
            "Mean Intensity Total": 0.0,
            "Mean Intensity Opacity": 0.0,
            "Std Dev Total": 0.0,
            "Std Dev Opacity": 0.0
        }

    # ตัดเฉพาะในปอด
    diff_inside = diff_map * mask_binary
    img_inside = img_orig_norm * mask_binary
    
    # 1. Opacity (Threshold ปกติ)
    opacity_mask = (diff_inside > Config.THRESHOLD).astype(np.float32)
    opacity_pixels = np.sum(opacity_mask)
    
    # 2. High Opacity (Threshold สูง)
    high_opacity_mask = (diff_inside > Config.HIGH_OPACITY_THRESHOLD).astype(np.float32)
    high_opacity_pixels = np.sum(high_opacity_mask)
    
    # 3. Percentages
    pct_opacity = (opacity_pixels / lung_pixels) * 100
    pct_high_opacity = (high_opacity_pixels / lung_pixels) * 100
    
    # 4. Intensity Stats (Mapping HU -> Intensity 0-255)
    # Mean Total (ทั้งปอด)
    mean_intensity_total = np.sum(img_inside) / lung_pixels * 255
    
    # Mean Opacity (เฉพาะส่วนฝ้า)
    if opacity_pixels > 0:
        mean_intensity_opacity = np.sum(img_inside * opacity_mask) / opacity_pixels * 255
    else:
        mean_intensity_opacity = 0.0
        
    # Std Dev Total
    # ต้องดึงค่าพิกเซลออกมาคำนวณ std (เฉพาะ pixel ที่อยู่ใน mask)
    pixels_in_lung = img_orig_norm[mask_binary == 1]
    std_total = np.std(pixels_in_lung * 255) if len(pixels_in_lung) > 0 else 0
    
    # Std Dev Opacity
    pixels_in_opacity = img_orig_norm[opacity_mask == 1]
    std_opacity = np.std(pixels_in_opacity * 255) if len(pixels_in_opacity) > 0 else 0
    
    # 5. Opacity Score (Simple scaling 0-4 per region, similar to paper but simplified)
    # Paper: 0=0%, 1=<25%, 2=<50%, 3=<75%, 4=>75%
    if pct_opacity <= 1: op_score = 0
    elif pct_opacity <= 25: op_score = 1
    elif pct_opacity <= 50: op_score = 2
    elif pct_opacity <= 75: op_score = 3
    else: op_score = 4

    return {
        "Affected": "Yes" if pct_opacity > 1 else "No",
        "Opacity Score": op_score,
        "Lung Volume (px)": int(lung_pixels),
        "Volume of Opacity (px)": int(opacity_pixels),
        "% Opacity": round(pct_opacity, 2),
        "Volume High Opacity (px)": int(high_opacity_pixels),
        "% High Opacity": round(pct_high_opacity, 2),
        "Mean Intensity Total": round(mean_intensity_total, 2),
        "Mean Intensity Opacity": round(mean_intensity_opacity, 2),
        "Std Dev Total": round(std_total, 2),
        "Std Dev Opacity": round(std_opacity, 2)
    }

# ================= 4. CORE FUNCTION =================
def analyze_lesion_score(img_path, model, scheduler, start_timestep=180, threshold=0.15, seed=42):
    # รับ Mask แยกซ้ายขวามาด้วย
    input_tensor, original_img_np, mask_total, mask_l, mask_r = process_image(img_path)
    if input_tensor is None: return None, None, None, None, None

    # --- Locus ---
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

    # Diffusion Process
    noise = torch.randn(input_tensor.shape).to(device)
    timesteps = torch.tensor([start_timestep], device=device).long()
    noisy_image = scheduler.add_noise(input_tensor, noise, timesteps)

    current_image = noisy_image
    scheduler_timesteps = scheduler.timesteps
    start_index = (scheduler_timesteps == start_timestep).nonzero(as_tuple=True)[0].item()
    subset_timesteps = scheduler_timesteps[start_index:]

    for t in subset_timesteps:
        with torch.no_grad():
            model_output = model(current_image, t).sample
            current_image = scheduler.step(model_output, t, current_image).prev_sample

    # Prepare Images
    img_recon = current_image.cpu().numpy()[0, 0]
    img_orig = input_tensor.cpu().numpy()[0, 0]
    
    img_orig = (img_orig + 1) / 2
    img_recon = (img_recon - img_recon.min()) / (img_recon.max() - img_recon.min())
    
    img_orig_blur = cv2.GaussianBlur(img_orig, (3, 3), 0)
    diff_raw = np.abs(img_orig_blur - img_recon)
    
    # --- 📊 CALCULATE METRICS (L/R/Total) ---
    metrics_total = calculate_lung_metrics(diff_raw, mask_total, img_orig)
    metrics_left = calculate_lung_metrics(diff_raw, mask_l, img_orig)
    metrics_right = calculate_lung_metrics(diff_raw, mask_r, img_orig)
    
    # Score หลัก (MAE) สำหรับใช้ในกราฟ
    lung_pixel_count = metrics_total["Lung Volume (px)"]
    if lung_pixel_count == 0: score_mae = 0.0
    else: 
        diff_inside = diff_raw * mask_total
        score_mae = (np.sum(diff_inside) / lung_pixel_count) * 100

    # --- Overlay Generation ---
    diff_inside_lung = diff_raw * mask_total
    lesion_map = np.where(diff_inside_lung > threshold, diff_inside_lung, 0)
    
    heatmap_display = np.clip(lesion_map * 3, 0, 1)
    heatmap_color = cv2.applyColorMap((heatmap_display * 255).astype(np.uint8), cv2.COLORMAP_JET)
    orig_bgr = cv2.cvtColor((img_orig * 255).astype(np.uint8), cv2.COLOR_GRAY2BGR)
    
    mask_lesion = (heatmap_display > 0).astype(np.uint8) * 255
    bg = cv2.bitwise_and(orig_bgr, orig_bgr, mask=cv2.bitwise_not(mask_lesion))
    fg = cv2.bitwise_and(heatmap_color, heatmap_color, mask=mask_lesion)
    overlay = cv2.add(bg, fg)

    # BG สีแดงเข้ม #800000
    outside_mask = (mask_total == 0)
    overlay[outside_mask, 0] = 0
    overlay[outside_mask, 1] = 0
    overlay[outside_mask, 2] = 128

    return overlay, score_mae, img_orig, img_recon, (metrics_total, metrics_left, metrics_right)

# ================= 5. COMPARISON FUNCTION =================
def generate_comparison_plot(res1, res2, pid, cls_name, dates):
    overlay1, score1, orig1, recon1, _ = res1
    overlay2, score2, orig2, recon2, _ = res2
    
    diff_score = score2 - score1
    
    if diff_score < 0: status = "อาการดีขึ้น (Improved)"; color = 'green'
    elif diff_score > 0: status = "อาการแย่ลง (Worsened)"; color = 'red'
    else: status = "อาการทรงตัว (Stable)"; color = 'blue'

    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    # Day 1
    axes[0, 0].imshow(orig1, cmap='gray'); axes[0, 0].set_title(f"Day 1 ({dates[0]})", fontsize=12)
    axes[0, 1].imshow(recon1, cmap='gray'); axes[0, 1].set_title("Day 1: AI Healed", fontsize=12)
    axes[0, 2].imshow(cv2.cvtColor(overlay1, cv2.COLOR_BGR2RGB)) 
    axes[0, 2].set_title(f"Score: {score1:.2f} (MAE)", fontsize=12, fontweight='bold', color='blue')
    
    # Day Last
    axes[1, 0].imshow(orig2, cmap='gray'); axes[1, 0].set_title(f"Day Last ({dates[1]})", fontsize=12)
    axes[1, 1].imshow(recon2, cmap='gray'); axes[1, 1].set_title("Day Last: AI Healed", fontsize=12)
    axes[1, 2].imshow(cv2.cvtColor(overlay2, cv2.COLOR_BGR2RGB)) 
    axes[1, 2].set_title(f"Score: {score2:.2f} (MAE)", fontsize=12, fontweight='bold', color='blue')
    
    for ax in axes.flat: ax.axis('off')
    plt.suptitle(f"Patient {pid} Progress: {status} (Diff: {diff_score:+.2f})", fontsize=16, color=color, y=1)
    plt.tight_layout()
    
    save_path = os.path.join(Config.DIRS["comparisons"], f"{cls_name}_{pid}_Compare.jpg")
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.close()

# ================= 6. EXECUTION LOOP =================
def get_timeline_files(folder):
    files = glob.glob(os.path.join(folder, "*.jpg"))
    daily = {}
    for f in files:
        try:
            parts = os.path.basename(f).replace(".jpg", "").split("_")
            d = datetime.strptime(parts[-2], "%Y%m%d")
            s = int(parts[-1])
            if d not in daily or s > daily[d][0]: daily[d] = (s, f)
        except: continue
    return [daily[d][1] for d in sorted(daily)], sorted(daily)

from datetime import datetime

all_csv_data = [] # List สำหรับเก็บข้อมูล CSV แบบละเอียด
print("🚀 Starting Analysis with FULL CSV METRICS...")

for cls in ["novap", "vap"]:
    c_path = os.path.join(Config.ROOT_DIR, cls)
    if not os.path.exists(c_path): continue
    
    for pid in os.listdir(c_path):
        p_path = os.path.join(c_path, pid)
        files, dates = get_timeline_files(p_path)
        if not files: continue
        
        print(f"Processing: {pid} ({len(files)} scans)")
        patient_results = []
        patient_dates = []
        mae_scores = []
        
        # 1. Analyze Each Image
        for i, f in enumerate(files):
            date_s = dates[i].strftime("%Y-%m-%d")
            
            # เรียกใช้ฟังก์ชันที่ปรับปรุงแล้ว
            res = analyze_lesion_score(f, diffusion_model, noise_scheduler, 
                                       start_timestep=Config.START_TIMESTEP, 
                                       threshold=Config.THRESHOLD,
                                       seed=Config.SEED)
            
            overlay, score_mae, orig, recon, detailed_metrics = res
            
            if overlay is not None:
                patient_results.append(res)
                patient_dates.append(date_s)
                mae_scores.append(score_mae)
                
                # Unpack metrics
                m_total, m_left, m_right = detailed_metrics
                
                # Save Heatmap
                fname = f"{pid}_{date_s}.jpg"
                save_dir = os.path.join(Config.DIRS["heatmaps"], cls, pid)
                if not os.path.exists(save_dir): os.makedirs(save_dir)
                cv2.imwrite(os.path.join(save_dir, fname), overlay)
                
                # --- PREPARE CSV ROW DATA ---
                row = {
                    "Patient_ID": pid,
                    "Date": date_s,
                    "Group": cls,
                    
                    # --- Total Overview ---
                    "Total Opacity Score (Sum L+R)": m_left["Opacity Score"] + m_right["Opacity Score"],
                    "Total % Opacity": m_total["% Opacity"],
                    
                    # --- Both Lungs ---
                    "Both_Affected": m_total["Affected"],
                    "Both_Opacity_Score": m_total["Opacity Score"], # Score รวมแบบ MAE mapping
                    "Both_Lung_Volume_px": m_total["Lung Volume (px)"],
                    "Both_Volume_Opacity_px": m_total["Volume of Opacity (px)"],
                    "Both_Pct_Opacity": m_total["% Opacity"],
                    "Both_Volume_High_Opacity_px": m_total["Volume High Opacity (px)"],
                    "Both_Pct_High_Opacity": m_total["% High Opacity"],
                    "Both_Mean_Intensity": m_total["Mean Intensity Total"],
                    "Both_Mean_Intensity_Opacity": m_total["Mean Intensity Opacity"],
                    "Both_StdDev_Total": m_total["Std Dev Total"],
                    "Both_StdDev_Opacity": m_total["Std Dev Opacity"],

                    # --- Left Lung ---
                    "Left_Affected": m_left["Affected"],
                    "Left_Opacity_Score": m_left["Opacity Score"],
                    "Left_Lung_Volume_px": m_left["Lung Volume (px)"],
                    "Left_Volume_Opacity_px": m_left["Volume of Opacity (px)"],
                    "Left_Pct_Opacity": m_left["% Opacity"],
                    "Left_Volume_High_Opacity_px": m_left["Volume High Opacity (px)"],
                    "Left_Pct_High_Opacity": m_left["% High Opacity"],
                    "Left_Mean_Intensity": m_left["Mean Intensity Total"],
                    "Left_Mean_Intensity_Opacity": m_left["Mean Intensity Opacity"],
                    "Left_StdDev_Total": m_left["Std Dev Total"],
                    "Left_StdDev_Opacity": m_left["Std Dev Opacity"],

                    # --- Right Lung ---
                    "Right_Affected": m_right["Affected"],
                    "Right_Opacity_Score": m_right["Opacity Score"],
                    "Right_Lung_Volume_px": m_right["Lung Volume (px)"],
                    "Right_Volume_Opacity_px": m_right["Volume of Opacity (px)"],
                    "Right_Pct_Opacity": m_right["% Opacity"],
                    "Right_Volume_High_Opacity_px": m_right["Volume High Opacity (px)"],
                    "Right_Pct_High_Opacity": m_right["% High Opacity"],
                    "Right_Mean_Intensity": m_right["Mean Intensity Total"],
                    "Right_Mean_Intensity_Opacity": m_right["Mean Intensity Opacity"],
                    "Right_StdDev_Total": m_right["Std Dev Total"],
                    "Right_StdDev_Opacity": m_right["Std Dev Opacity"],
                }
                all_csv_data.append(row)

        # 2. Compare First vs Last
        if len(patient_results) >= 2:
            generate_comparison_plot(patient_results[0], patient_results[-1], pid, cls, [patient_dates[0], patient_dates[-1]])
        
        # 3. Timeline Plot
        if len(patient_results) > 0:
            plt.figure(figsize=(4 * len(patient_results), 8))
            for i in range(len(patient_results)):
                ax = plt.subplot(2, len(patient_results), i+1)
                ax.imshow(cv2.cvtColor(patient_results[i][0], cv2.COLOR_BGR2RGB))
                
                current_score = mae_scores[i]
                if i == 0: color = 'blue'
                else:
                    prev_score = mae_scores[i-1]
                    if current_score < prev_score: color = 'green'
                    elif current_score > prev_score: color = 'red'
                    else: color = 'blue'
                
                ax.set_title(f"{patient_dates[i]}\nScore: {current_score:.2f}", color=color, fontweight='bold', fontsize=14)
                ax.axis('off')
            
            ax2 = plt.subplot(2, 1, 2)
            ax2.plot(patient_dates, mae_scores, 'b-o', linewidth=2)
            ax2.set_ylabel("Opacity Score (MAE)")
            ax2.grid(True, alpha=0.3)
            
            plt.suptitle(f"Timeline: {pid} ({cls})", fontsize=16)
            plt.savefig(os.path.join(Config.DIRS["timelines"], f"{cls}_{pid}_Timeline.jpg"), bbox_inches='tight')
            plt.close()

# Save CSV with Full Metrics
if all_csv_data:
    df = pd.DataFrame(all_csv_data)
    
    # จัดเรียง Column ให้สวยงามตามที่ขอ
    cols = ["Patient_ID", "Date", "Group", "Total Opacity Score (Sum L+R)", "Total % Opacity"]
    # เพิ่ม Column ของ Both/Left/Right ต่อท้าย
    cols += [c for c in df.columns if c.startswith("Both_")]
    cols += [c for c in df.columns if c.startswith("Left_")]
    cols += [c for c in df.columns if c.startswith("Right_")]
    
    df = df[cols]
    
    csv_path = os.path.join(Config.DIRS["csv"], "Clinical_Data.csv")
    df.to_csv(csv_path, index=False)
    print(f"\n✅ Final Clinical Report Saved: {csv_path}")
    print("   Note: Volume is in 'pixels' and Intensity is '0-255' (mapped from CT methods for X-ray)")

⏳ Loading Models...
✅ Models Ready!
🚀 Starting Analysis with FULL CSV METRICS...
Processing: 2268 (10 scans)


In [1]:
import os
import glob
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torchxrayvision as xrv
from diffusers import DDPMScheduler, UNet2DModel
from tqdm.auto import tqdm
import warnings
from datetime import datetime

warnings.filterwarnings("ignore")
plt.rcParams['font.family'] = 'Tahoma'

device = "cuda" if torch.cuda.is_available() else "cpu"

# ================= 1. CONFIGURATION =================
class Config:
    ROOT_DIR = "Chest xray CP class"
    MODEL_PATH = "lung_diffusion_model"
    
    BASE_OUTPUT = "Final_Paper_Based_Heatmap" 
    DIRS = {
        "heatmaps": os.path.join(BASE_OUTPUT, "Heatmaps"),
        "timelines": os.path.join(BASE_OUTPUT, "Timelines"),
        "comparisons": os.path.join(BASE_OUTPUT, "Comparisons"),
        "csv": os.path.join(BASE_OUTPUT, "Clinical_Data")
    }
    
    IMG_SIZE = 128
    START_TIMESTEP = 180 
    
    # Thresholds
    HEATMAP_THRESHOLD = 0.10 
    CONTOUR_THRESHOLD = 0.25 
    
    SEED = 42

for d in Config.DIRS.values():
    if not os.path.exists(d): os.makedirs(d)

# ================= 2. MODEL LOADING =================
print("⏳ Loading Models...")
seg_model = xrv.baseline_models.chestx_det.PSPNet()
seg_model.eval()

if os.path.exists(Config.MODEL_PATH):
    noise_scheduler = DDPMScheduler(num_train_timesteps=1000)
    diffusion_model = UNet2DModel.from_pretrained(Config.MODEL_PATH).to(device)
    diffusion_model.eval()
    print("✅ Models Ready!")
else:
    raise FileNotFoundError("❌ ไม่พบโมเดล Diffusion")

# ================= 3. HELPER FUNCTIONS =================
def get_masks(img_numpy_uint8):
    try:
        h, w = img_numpy_uint8.shape
        img_norm = xrv.datasets.normalize(img_numpy_uint8, 255) 
        img_resized = cv2.resize(img_norm, (512, 512))
        img_tensor = torch.from_numpy(img_resized)[None, None, ...].float()
        with torch.no_grad(): outputs = seg_model(img_tensor)
        pred = outputs[0].numpy()
        def resize_m(m): return cv2.resize(m, (w, h), interpolation=cv2.INTER_LINEAR)
        
        mask_l = (resize_m(pred[4]) > 0.5).astype(np.uint8) * 255
        mask_r = (resize_m(pred[5]) > 0.5).astype(np.uint8) * 255
        mask_total = cv2.bitwise_or(mask_l, mask_r)
        
        return mask_total, mask_l, mask_r
    except:
        z = np.zeros(img_numpy_uint8.shape, dtype=np.uint8)
        return z, z, z

def enhance_lung_clarity(img_gray, lung_mask):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    enhanced = clahe.apply(img_gray)
    return cv2.bitwise_and(enhanced, enhanced, mask=lung_mask)

def process_image(img_path):
    if not os.path.exists(img_path): return None, None, None, None, None
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is None: return None, None, None, None, None
    
    mask_total, mask_l, mask_r = get_masks(img)
    
    def resize_binary(m):
        m_res = cv2.resize(m, (Config.IMG_SIZE, Config.IMG_SIZE), interpolation=cv2.INTER_NEAREST)
        return (m_res > 127).astype(np.float32)

    mask_binary_total = resize_binary(mask_total)
    mask_binary_l = resize_binary(mask_l)
    mask_binary_r = resize_binary(mask_r)

    if np.sum(mask_total) == 0: enhanced = img
    else: enhanced = enhance_lung_clarity(img, mask_total)
        
    resized = cv2.resize(enhanced, (Config.IMG_SIZE, Config.IMG_SIZE))
    img_tensor = torch.tensor(resized).float() / 255.0
    
    return img_tensor.unsqueeze(0).unsqueeze(0).to(device), img, mask_binary_total, mask_binary_l, mask_binary_r

# ================= 3.1 UPDATED METRICS CALCULATION =================
def calculate_lung_metrics(diff_map, mask_binary, img_orig_norm):
    """
    diff_map: แผนที่ความต่าง (Difference Map)
    mask_binary: หน้ากากปอดส่วนที่สนใจ (0 หรือ 1)
    img_orig_norm: ภาพต้นฉบับที่ Normalize แล้ว (0.0 - 1.0)
    """
    # 1. Basic Volume
    lung_pixels = np.sum(mask_binary)
    if lung_pixels == 0:
        return {
            "Affected": "No", "Opacity_Score": 0, "Lung_Volume_px": 0, "Volume_Opacity_px": 0, 
            "Pct_Opacity": 0.0, "Volume_High_Opacity_px": 0, "Pct_High_Opacity": 0.0,
            "Mean_Intensity": 0.0, "Mean_Intensity_Opacity": 0.0, "StdDev_Total": 0.0, "StdDev_Opacity": 0.0
        }

    # 2. Opacity (General) - ใช้ HEATMAP_THRESHOLD
    diff_inside = diff_map * mask_binary
    opacity_mask = (diff_inside > Config.HEATMAP_THRESHOLD).astype(np.float32)
    opacity_pixels = np.sum(opacity_mask)
    pct_opacity = (opacity_pixels / lung_pixels) * 100
    
    # 3. High Opacity (Severe) - ใช้ CONTOUR_THRESHOLD
    high_opacity_mask = (diff_inside > Config.CONTOUR_THRESHOLD).astype(np.float32)
    high_opacity_pixels = np.sum(high_opacity_mask)
    pct_high_opacity = (high_opacity_pixels / lung_pixels) * 100

    # 4. Opacity Score logic
    if pct_opacity <= 1: op_score = 0
    elif pct_opacity <= 25: op_score = 1
    elif pct_opacity <= 50: op_score = 2
    elif pct_opacity <= 75: op_score = 3
    else: op_score = 4

    # 5. Intensity & StdDev Stats
    # เอาพิกเซลภายในปอดทั้งหมดออกมาเพื่อหา Mean/Std ของ "เนื้อปอดเดิม"
    lung_values = img_orig_norm[mask_binary > 0]
    mean_intensity = np.mean(lung_values) if len(lung_values) > 0 else 0
    std_total = np.std(lung_values) if len(lung_values) > 0 else 0

    # เอาพิกเซลเฉพาะตรงที่เป็นรอยโรค (จาก Diff Map หรือ Original ก็ได้ แต่โจทย์ถามความหนาแน่นฝ้า มักดูที่ Diff Map หรือ Original ตรงฝ้า)
    # *ในที่นี้ขอใช้ค่าจาก Difference Map เพื่อสื่อถึงความรุนแรงของฝ้าที่ AI เห็น*
    opacity_values = diff_inside[opacity_mask > 0]
    mean_int_op = np.mean(opacity_values) if len(opacity_values) > 0 else 0
    std_op = np.std(opacity_values) if len(opacity_values) > 0 else 0

    return {
        "Affected": "Yes" if pct_opacity > 1 else "No",
        "Opacity_Score": op_score,
        "Lung_Volume_px": int(lung_pixels),
        "Volume_Opacity_px": int(opacity_pixels),
        "Pct_Opacity": round(pct_opacity, 2),
        "Volume_High_Opacity_px": int(high_opacity_pixels),
        "Pct_High_Opacity": round(pct_high_opacity, 2),
        "Mean_Intensity": round(mean_intensity, 4),           # ความสว่างเฉลี่ยปอด
        "Mean_Intensity_Opacity": round(mean_int_op, 4),      # ความสว่างเฉลี่ยรอยโรค
        "StdDev_Total": round(std_total, 4),                  # StdDev ปอดรวม
        "StdDev_Opacity": round(std_op, 4)                    # StdDev รอยโรค
    }

# ================= 4. CORE FUNCTION =================
def analyze_lesion_score(img_path, model, scheduler, start_timestep=180, seed=42):
    # 1. Process
    input_tensor, original_img_np, mask_total, mask_l, mask_r = process_image(img_path)
    if input_tensor is None: return None, None, None, None, None

    # Locus Seed
    torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

    # Diffusion
    noise = torch.randn(input_tensor.shape).to(device)
    timesteps = torch.tensor([start_timestep], device=device).long()
    noisy_image = scheduler.add_noise(input_tensor, noise, timesteps)

    current_image = noisy_image
    scheduler_timesteps = scheduler.timesteps
    start_index = (scheduler_timesteps == start_timestep).nonzero(as_tuple=True)[0].item()
    subset_timesteps = scheduler_timesteps[start_index:]

    for t in subset_timesteps:
        with torch.no_grad():
            model_output = model(current_image, t).sample
            current_image = scheduler.step(model_output, t, current_image).prev_sample

    # Prepare Images
    img_recon = current_image.cpu().numpy()[0, 0]
    img_orig = input_tensor.cpu().numpy()[0, 0]
    img_orig = (img_orig + 1) / 2
    img_recon = (img_recon - img_recon.min()) / (img_recon.max() - img_recon.min())
    
    img_orig_blur = cv2.GaussianBlur(img_orig, (3, 3), 0)
    
    # Diff Calculation
    diff_raw = np.abs(img_orig_blur - img_recon)
    diff_clean = diff_raw * mask_total

    # Calculate Detailed Metrics
    metrics_total = calculate_lung_metrics(diff_raw, mask_total, img_orig)
    metrics_left = calculate_lung_metrics(diff_raw, mask_l, img_orig)
    metrics_right = calculate_lung_metrics(diff_raw, mask_r, img_orig)
    
    # Score Calculation (MAE)
    lung_pixel_count = metrics_total["Lung_Volume_px"]
    if lung_pixel_count == 0: score_mae = 0.0
    else: score_mae = (np.sum(diff_clean) / lung_pixel_count) * 100

    # ================= VISUALIZATION =================
    max_diff = np.max(diff_clean)
    norm_diff = (diff_clean / max_diff) if max_diff > 0 else diff_clean
    norm_diff = np.where(norm_diff < Config.HEATMAP_THRESHOLD, 0, norm_diff)

    heatmap_uint8 = (norm_diff * 255).astype(np.uint8)
    heatmap_color = cv2.applyColorMap(heatmap_uint8, cv2.COLORMAP_JET)
    
    orig_uint8 = (img_orig * 255).astype(np.uint8)
    orig_bgr = cv2.cvtColor(orig_uint8, cv2.COLOR_GRAY2BGR)

    heatmap_mask = (norm_diff > 0).astype(np.float32)[:, :, None]
    overlay = orig_bgr.astype(np.float32) * (1 - heatmap_mask * 0.4) + \
              heatmap_color.astype(np.float32) * (heatmap_mask * 0.4)
    overlay = overlay.astype(np.uint8)

    # Contours
    high_opacity_map = (norm_diff > Config.CONTOUR_THRESHOLD).astype(np.uint8)
    contours, _ = cv2.findContours(high_opacity_map, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(overlay, contours, -1, (0, 255, 255), 1) 

    # --- 🛠️ FIX: Change Background to Black ---
    outside_mask = (mask_total == 0)
    overlay[outside_mask] = 0 # เปลี่ยนเป็นสีดำสนิท [0,0,0]

    return overlay, score_mae, img_orig, img_recon, (metrics_total, metrics_left, metrics_right)

# ================= 5. COMPARISON FUNCTION =================
def generate_comparison_plot(res1, res2, pid, cls_name, dates):
    overlay1, score1, orig1, recon1, _ = res1
    overlay2, score2, orig2, recon2, _ = res2
    
    diff_score = score2 - score1
    if diff_score < 0: status = "อาการดีขึ้น (Improved)"; color = 'green'
    elif diff_score > 0: status = "อาการแย่ลง (Worsened)"; color = 'red'
    else: status = "อาการทรงตัว (Stable)"; color = 'blue'

    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    axes[0, 0].imshow(orig1, cmap='gray'); axes[0, 0].set_title(f"Day 1 ({dates[0]})", fontsize=12)
    axes[0, 1].imshow(recon1, cmap='gray'); axes[0, 1].set_title("Day 1: AI Reconstructed", fontsize=12)
    axes[0, 2].imshow(cv2.cvtColor(overlay1, cv2.COLOR_BGR2RGB)) 
    axes[0, 2].set_title(f"Score: {score1:.2f} (MAE)", fontsize=12, fontweight='bold', color='blue')
    
    axes[1, 0].imshow(orig2, cmap='gray'); axes[1, 0].set_title(f"Day Last ({dates[1]})", fontsize=12)
    axes[1, 1].imshow(recon2, cmap='gray'); axes[1, 1].set_title("Day Last: AI Reconstructed", fontsize=12)
    axes[1, 2].imshow(cv2.cvtColor(overlay2, cv2.COLOR_BGR2RGB)) 
    axes[1, 2].set_title(f"Score: {score2:.2f} (MAE)", fontsize=12, fontweight='bold', color='blue')
    
    for ax in axes.flat: ax.axis('off')
    plt.suptitle(f"Patient {pid} Progress: {status} (Diff: {diff_score:+.2f})", fontsize=16, color=color, y=1)
    plt.tight_layout()
    
    save_path = os.path.join(Config.DIRS["comparisons"], f"{cls_name}_{pid}_Compare.jpg")
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.close()

# ================= 6. EXECUTION LOOP =================
def get_timeline_files(folder):
    files = glob.glob(os.path.join(folder, "*.jpg"))
    daily = {}
    for f in files:
        try:
            parts = os.path.basename(f).replace(".jpg", "").split("_")
            d = datetime.strptime(parts[-2], "%Y%m%d")
            s = int(parts[-1])
            # เก็บอันที่ sequence มากสุด (ล่าสุดของวัน)
            if d not in daily or s > daily[d][0]: daily[d] = (s, f)
        except: continue
    return [daily[d][1] for d in sorted(daily)], sorted(daily)

all_csv_data = [] 
print("🚀 Starting Analysis with Detailed CSV & Black Background...")

for cls in ["novap", "vap"]:
    c_path = os.path.join(Config.ROOT_DIR, cls)
    if not os.path.exists(c_path): continue
    
    for pid in os.listdir(c_path):
        p_path = os.path.join(c_path, pid)
        files, dates = get_timeline_files(p_path)
        if not files: continue
        
        print(f"Processing: {pid} ({len(files)} scans)")
        patient_results = []
        patient_dates = []
        mae_scores = []
        
        # Dictionary to track duplicate dates for display
        date_counts = {}

        for i, f in enumerate(files):
            # จัดการ Date String ถ้าซ้ำ (แม้ get_timeline_files จะกรองแล้ว แต่ทำเผื่อไว้)
            d_obj = dates[i]
            base_date_s = d_obj.strftime("%Y-%m-%d")
            
            if base_date_s not in date_counts:
                date_counts[base_date_s] = 1
                final_date_str = base_date_s
            else:
                date_counts[base_date_s] += 1
                final_date_str = f"{base_date_s}({date_counts[base_date_s]})"

            res = analyze_lesion_score(f, diffusion_model, noise_scheduler, start_timestep=Config.START_TIMESTEP, seed=Config.SEED)
            
            overlay, score_mae, orig, recon, detailed_metrics = res
            
            if overlay is not None:
                patient_results.append(res)
                patient_dates.append(final_date_str)
                mae_scores.append(score_mae)
                
                m_total, m_left, m_right = detailed_metrics
                
                # Save Image
                fname = f"{pid}_{final_date_str.replace('/', '-')}.jpg"
                save_dir = os.path.join(Config.DIRS["heatmaps"], cls, pid)
                if not os.path.exists(save_dir): os.makedirs(save_dir)
                cv2.imwrite(os.path.join(save_dir, fname), overlay)
                
                # --- PREPARE CSV ROW ---
                row = {
                    "Patient_ID": pid,
                    "Date": final_date_str,
                    "Group": cls,
                    
                    # 2. Overall Scores
                    "Total Opacity Score (Sum L+R)": m_left["Opacity_Score"] + m_right["Opacity_Score"],
                    "Total % Opacity": m_total["Pct_Opacity"],
                }

                # 3. Add Detailed Metrics (Flattening dicts)
                # Helper function to add prefix
                def add_metrics(prefix, m_dict):
                    for k, v in m_dict.items():
                        row[f"{prefix}_{k}"] = v
                
                add_metrics("Both", m_total)
                add_metrics("Left", m_left)
                add_metrics("Right", m_right)
                
                all_csv_data.append(row)

        # Timeline generation (Same as before)
        if len(patient_results) >= 2:
            generate_comparison_plot(patient_results[0], patient_results[-1], pid, cls, [patient_dates[0], patient_dates[-1]])
        
        if len(patient_results) > 0:
            plt.figure(figsize=(4 * len(patient_results), 8))
            for i in range(len(patient_results)):
                ax = plt.subplot(2, len(patient_results), i+1)
                ax.imshow(cv2.cvtColor(patient_results[i][0], cv2.COLOR_BGR2RGB))
                
                current_score = mae_scores[i]
                if i == 0: color = 'blue'
                else:
                    prev_score = mae_scores[i-1]
                    if current_score < prev_score: color = 'green'
                    elif current_score > prev_score: color = 'red'
                    else: color = 'blue'
                ax.set_title(f"{patient_dates[i]}\nScore: {current_score:.2f}", color=color, fontweight='bold', fontsize=14)
                ax.axis('off')
            
            ax2 = plt.subplot(2, 1, 2)
            ax2.plot(patient_dates, mae_scores, 'b-o', linewidth=2)
            ax2.set_ylabel("Opacity Score (MAE)")
            ax2.grid(True, alpha=0.3)
            plt.suptitle(f"Timeline: {pid} ({cls})", fontsize=16)
            plt.savefig(os.path.join(Config.DIRS["timelines"], f"{cls}_{pid}_Timeline.jpg"), bbox_inches='tight')
            plt.close()

if all_csv_data:
    df = pd.DataFrame(all_csv_data)
    
    # จัดลำดับ Column ให้สวยงามตามที่ขอ (Optional sorting)
    first_cols = ["Patient_ID", "Date", "Group", "Total Opacity Score (Sum L+R)", "Total % Opacity"]
    other_cols = [c for c in df.columns if c not in first_cols]
    # เรียง other_cols ให้ Both มาก่อน Left มาก่อน Right
    other_cols.sort(key=lambda x: (0 if "Both" in x else 1 if "Left" in x else 2, x))
    
    df = df[first_cols + other_cols]

    csv_path = os.path.join(Config.DIRS["csv"], "Clinical_Data_Detailed.csv")
    df.to_csv(csv_path, index=False)
    print(f"\n✅ Final Clinical Report Saved: {csv_path}")

⏳ Loading Models...
✅ Models Ready!
🚀 Starting Analysis with Detailed CSV & Black Background...
Processing: 2268 (10 scans)
Processing: 2272 (18 scans)
Processing: 2273 (4 scans)
Processing: 2278 (5 scans)
Processing: 2279 (6 scans)
Processing: 2283 (6 scans)
Processing: 2285 (5 scans)
Processing: 2286 (9 scans)
Processing: 2288 (5 scans)
Processing: 2289 (6 scans)
Processing: 2290 (13 scans)
Processing: 2299 (2 scans)
Processing: 2300 (10 scans)
Processing: 2303 (3 scans)
Processing: 2304 (6 scans)
Processing: 2310 (1 scans)
Processing: 2312 (5 scans)
Processing: 2315 (5 scans)
Processing: 2316 (2 scans)
Processing: 2317 (7 scans)
Processing: 2320 (2 scans)
Processing: 2321 (9 scans)
Processing: 2328 (8 scans)
Processing: 2336 (6 scans)
Processing: 2337 (4 scans)
Processing: 2338 (4 scans)
Processing: 2131 (7 scans)
Processing: 2140 (25 scans)
Processing: 2157 (18 scans)
Processing: 2167 (14 scans)
Processing: 2179 (26 scans)
Processing: 2199 (14 scans)
Processing: 2200 (13 scans)
Pro